In [ ]:
# !pip install pytorch-lightning

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import random_split
# import pytorch_lightning as pl

In [ ]:
# Data load
batch_size = 32

train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
# Model
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [ ]:
# Hyperparameters and model initialization
# Testing it with one epoch to and analyzing results.
# Current settings provide 91.19% Training Accuracy and 91.52% Testing Accuracy 
num_epochs = 1
input_dim = 784
output_dim = 10
lr_rate = 0.1

model = LogisticRegression(input_dim, output_dim)


In [ ]:
# Loss
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
# Train 
for epoch in range(int(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, input_dim)

        outputs = model(images)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        if i%500==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = images.reshape(-1, input_dim)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)

                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()

            accuracy = 100 * correct/total
            print("Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch+1, i, loss.item(), accuracy))

Epoch: 1. Iteration: 0. Loss: 2.3075413703918457. Accuracy: 28.200000762939453.
Epoch: 1. Iteration: 500. Loss: 0.311492383480072. Accuracy: 89.62000274658203.
Epoch: 1. Iteration: 1000. Loss: 0.17686724662780762. Accuracy: 90.79000091552734.
Epoch: 1. Iteration: 1500. Loss: 0.14751163125038147. Accuracy: 91.08999633789062.


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_dim)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the model on the 10000 test images: 91.41999816894531 %
